In [1]:
using PyCall
cma = pyimport("cma")

PyObject <module 'cma' from '/home/amjalled/.local/lib/python3.8/site-packages/cma/__init__.py'>

In [2]:
using DrWatson
@quickactivate :Catalyst

┌ Info: Precompiling Catalyst [01e5440a-74e3-4b45-b8b9-c13ddfc5051b]
└ @ Base loading.jl:1260


In [3]:
using Distributed
N = 20
addprocs(N)

20-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20
 21

In [4]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst

      From worker 10:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 5:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 2:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 14:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 12:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 18:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 7:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 21:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 6:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 19:	 Activating environment at `~/Repos/advection-diffusion-catalysi

In [5]:
function calibrate(cma,N)
    Init = [0.013235189689493238, 1., 1., 1., 1.] #initial guess
    opts = cma.CMAOptions()
    opts["bounds"] = [[0, 0, 0, 0, 0], [1, 50, 50, 30, 30]] 
    opts["popsize"] = N
    weights = cma.recombination_weights.RecombinationWeights(N)
    opts["CMA_recombination_weights"] = weights
    es = cma.CMAEvolutionStrategy(Init, 0.5, opts) 
    
    input_exp = []
    output_exp = []
    for row in CSV.File(datadir("experiment/SBA15_interp_8.csv"); delim = ",")
        push!(input_exp, row.I)
        push!(output_exp, row.O)
    end
    n = 2
    nels = (5n, convert(Int,2.5n),n)
    S = Vec((0.0,0.0,0.0))
    E = Vec((1.0,0.5,0.2))
    grid = generate_grid(Tetrahedron, nels, S, E)
    faces = union(getfaceset(grid,"top"),getfaceset(grid,"bottom"),getfaceset(grid,"left"),getfaceset(grid,"right"),
                   getfaceset(grid,"front"),getfaceset(grid,"back"))
    addfaceset!(grid, "1", faces)
    
    while isempty(es.stop()) 
        solutions = es.ask() 
        fitness = pmap(function eval(input_args) 
                            try 
                               return Catalyst.solve(input_args[1], input_args[2], input_args[3], 
                                          input_exp, output_exp, progress=false, 
                                          microcomp_type=:nonlinear, 
                                          Q=input_args[4], kₙ=input_args[5], 
                                          calibration=true,w=(1.5302e-4*(1/0.37)),micromesh=grid)
                             catch err #in case we loose newton convergence in a microscopic computation 
                                return 1e10
                             end
                        end
                            ,solutions) 
        es.tell(solutions,fitness) 
        es.logger.add() 
        es.disp() 
    end
end

calibrate (generic function with 1 method)

In [ ]:
@time calibrate(cma, N)

(10_w,20)-aCMA-ES (mu_w=5.9,w_1=27%) in dimension 5 (seed=566064, Fri Aug 14 17:39:51 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     20 4.804088234839943e+02 1.0e+00 6.40e-01  6e-01  7e-01 15:46.8
    2     40 4.356443049095802e+02 1.3e+00 9.24e-01  8e-01  1e+00 29:37.1
    3     60 3.346731060227774e+02 1.4e+00 1.73e+00  2e+00  2e+00 44:06.4
    4     80 2.909617301414613e+02 1.7e+00 2.25e+00  2e+00  3e+00 58:36.6
    5    100 2.000642318229738e+02 1.7e+00 2.78e+00  2e+00  3e+00 73:02.8
    6    120 1.441156562458977e+02 1.6e+00 3.15e+00  3e+00  3e+00 87:59.1
    7    140 7.569465645605483e+01 1.6e+00 3.22e+00  3e+00  4e+00 101:29.9
    8    160 2.418829249726345e+02 2.0e+00 3.33e+00  3e+00  4e+00 117:00.1
    9    180 8.687466044424635e+00 2.2e+00 3.65e+00  3e+00  4e+00 131:31.6
   10    200 3.180376114377782e+02 2.3e+00 4.37e+00  4e+00  4e+00 145:19.4
   11    220 2.442785717000811e+02 2.6e+00 5.03e+00  4e+00  6e+00 159:21.0
   12    240 3.06